<a href="https://colab.research.google.com/github/Zhiyuan-03/AI_in_Transportation_Exercise/blob/main/EdittedmodelEstimation_IDM_proj2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Course AH2174/FAH3002 - Model calibration using trajectory data    
Lab 3- Mini Project on Car-following Model    
Date: 11.27.2025

## Objective
Calibrating and validating IDM model by using real-world vehicle trajectory data.  

## Problem description
A general formulation of the CF model calibration problem is as follows:
![image.png](attachment:image.png)
*$β$* is a vector of car-following model parameters to be calibrated  
*$F(⋅)$* is a car-following model  
*$LB_β$* and *$UB_β$* represent the lower and the upper bound for the parameters in β, respectively  
*$G(⋅)$* is a vector of constraint functions
*$MoP^{obs}$* and *$MoP^{sim}$* represent observed and simulated measure of performance (MoP), respectively  
*$f(⋅)$* is a goodness-of-fit (GoF) function  
*$f(MoP^{obs}, MoP^{sim})$* is the optimization objective function

## IDM calibration
IDM model:
![image.png](attachment:image.png)
where *$a$* is maximum acceleration  
*$b$* is desired deceleration  
*$v_0$* is desired velocity  
*$S(t)$* is spacing  
*$v_n(t)$* is following vehicle velocity  
*$v_{n-1}(t)$* is preceding vehicle velocity  
*$T$* is the lower bound of the desired time gap  
*$s_0$* is jam distance  
*$δ$* is acceleration exponent, and typical value is 4  
*$S^*$* is desired spacing  

## Data description
The data file has the trajectory data of 25 car-following pairs. The frame rate is 10fps. Each vehicle has its unique vehicle id, and each row represents the data of the follower and the leader in a frame. The definitions of given fields are listed as follows:  
Frame_ID_f: The the sequential number of each frame  
Vehicle_ID_f: The vehicle id of the follower  
x_f (m): The position the follower center point  
v_f (m/s): The velocity of the follower  
a_f ($m/s^2$): The acceleration of the follower  
length_f (m): The length of the follower  
width_f (m): The width of the follwer  
Frame_ID_l: The the sequential number of each frame  
Vehicle_ID_l: The vehicle id of the leader car  
x_l (m): The position the leader center point  
v_l (m/s): The velocity of the leader  
a_l ($m/s^2$): The acceleration of the leader  
length_l (m): The length of the leader  
width_l (m): The width of the leader  
spacing (m): The distance between the front of the following vehicle and the rear of the leading vehicle

## Calibration
GoF: RMSE, NRMSE, MAE...  
MoP: spacing, position, speed...  
Optimization algorithm: Global least square (from scipy.optimize import least_squares), genetic algorithm (GA) (import pygad), Interior Point (IP)...  
Reference: Punzo, Vincenzo, Zuduo Zheng, and Marcello Montanino. "About calibration of car-following dynamics of automated and human-driven vehicles: Methodology, guidelines and codes." Transportation Research Part C: Emerging Technologies 128 (2021): 103165.

# Code
Complete the following parts by filling the code under "`YOU CODE HERE`". You can also change the rest of the code skeleton according to your need!

In [1]:
import numpy as np
import pandas as pd

In [5]:
from google.colab import files
import pandas as pd

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Read the uploaded CSV file
# Replace 'cf_pair25_v1.csv' with the actual filename you uploaded
data = pd.read_csv('cf_pair25_v1.csv')

# Step 3: Check the first few rows
print(data.head())


Saving cf_pair25_v1.csv to cf_pair25_v1.csv
   Frame_ID_f  Vehicle_ID_f        x_f       v_f       a_f  length_f  \
0      2147.0        1939.0   9.269805  5.175211 -0.139002  4.820432   
1      2148.0        1939.0   9.786631  5.167622 -0.139846  4.820432   
2      2149.0        1939.0  10.302694  5.149183 -0.140244  4.820432   
3      2150.0        1939.0  10.816911  5.121871 -0.139313  4.820432   
4      2151.0        1939.0  11.328401  5.094818 -0.137071  4.820432   

    width_f  Frame_ID_l  Vehicle_ID_l        x_l       v_l       a_l  \
0  2.246702      2147.0        1895.0  23.672291  5.090129 -0.182866   
1  2.246702      2148.0        1895.0  24.180390  5.068680 -0.182701   
2  2.246702      2149.0        1895.0  24.686344  5.038788 -0.181375   
3  2.246702      2150.0        1895.0  25.189316  5.004958 -0.178478   
4  2.246702      2151.0        1895.0  25.688920  4.976963 -0.174568   

   length_l  width_l   spacing  
0   5.10403  2.20983  9.440255  
1   5.10403  2.20983  9.

In [6]:
# 1. Load data
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
# Step 1: Upload the file (you already did this once, but keep for completeness)
uploaded = files.upload()

# Step 2: Load the uploaded CSV file
# Replace 'cf_pair25_v1.csv' with the exact filename you uploaded
data = pd.read_csv('cf_pair25_v1.csv')

# Split the data into a training set and a testing set in an 8:2 ratio.
# eg.20 car-following pairs for training, and 5 car-following pairs for testing
# YOU CODE HERE
####################################################################
# Step 3: Split the data into a training set and a testing set in an 8:2 ratio
unique_pairs = data['Vehicle_ID_f'].unique()

train_ids, test_ids = train_test_split(unique_pairs, test_size=0.2, random_state=42)

train_data = data[data['Vehicle_ID_f'].isin(train_ids)]
test_data = data[data['Vehicle_ID_f'].isin(test_ids)]

# Quick check
print("Training pairs:", len(train_ids))
print("Testing pairs:", len(test_ids))

Training pairs: 20
Testing pairs: 5


In [ ]:
# 2. Calibration

# Define the IDM model
def idm_model(s, v, delta_v, params, T = 1.5, v0 = 30, s0 = 1):

    a, b = params
    delta = 4  # Typical acceleration exponent for IDM

    #  YOU CODE HERE
    ####################################################################
    # Desired dynamic gap
    s_star = s0 + v * T + (v * delta_v) / (2 * (a * b) ** 0.5)

    # IDM acceleration formula
    acceleration = a * (1 - (v / v0) ** delta - (s_star / s) ** 2)
    return acceleration

# Define the Calibration error function
def calibration(params, df, time_step=0.1):

    # initial setup
    total_gof = 0
    vehicle_count = 0

    # Group by Vehicle_ID to process each following vehicle’s trajectory independently
    grouped = df.groupby('Vehicle_ID_f')

    for vehicle_id, data in grouped:

        # Initial state for the following vehicle
        #  YOU CODE HERE
        ####################################################################
        v_f = data['v_f'].iloc[0]       # initial speed of follower
        x_f = data['x_f'].iloc[0]       # initial position of follower
        a_f = data['a_f'].iloc[0]       # initial acceleration of follower

        # Leading vehicle's trajectory
        #  YOU CODE HERE
        ####################################################################
        v_l = data['v_l'].values        # speed trajectory of leader
        x_l = data['x_l'].values        # position trajectory of leader
        a_l = data['a_l'].values        # acceleration trajectory of leader
        # Observed trajectory of the following vehicle
        #  YOU CODE HERE
        ####################################################################
        v_obs = data['v_f'].values      # observed follower speeds
        x_obs = data['x_f'].values      # observed follower positions
        a_obs = data['a_f'].values      # observed follower accelerations
        # Initialize lists to store the simulated trajectory
        #  YOU CODE HERE
        ####################################################################
        v_sim = [v_f]                   # simulated follower speed
        x_sim = [x_f]                   # simulated follower position
        a_sim = [a_f]                   # simulated follower acceleration
        # Step through the leader’s trajectory and simulate the following vehicle’s trajectory
        for i in range(1, len(data)):

            #  YOU CODE HERE
            ####################################################################
        s = x_l[i] - x_sim[-1] - data['length_l'].iloc[i]  # gap
        delta_v = v_sim[-1] - v_l[i]                       # relative speed
        acc = idm_model(s, v_sim[-1], delta_v, params)     # IDM acceleration
        new_v = v_sim[-1] + acc * time_step
        new_x = x_sim[-1] + new_v * time_step

        v_sim.append(new_v)
        x_sim.append(new_x)
        a_sim.append(acc)


        # Calculate GoF between the observed and predicted MoP for this vehicle
        #  YOU CODE HERE
        ####################################################################

# Calculate GoF between the observed and predicted MoP for this vehicle
# Example: RMSE for speed trajectory
       gof = np.sqrt(np.mean((v_obs - np.array(v_sim))**2))

       total_gof += gof  # Accumulate RMSE
       vehicle_count += 1

     # Return the average MSE
      return total_gof / vehicle_count

# Initial guess for IDM parameters: a, b
initial_guess =

# Define bounds to ensure non-negative parameters for a, and b
bounds =

# Run optimization algorithm for parameter calibration
#  YOU CODE HERE
####################################################################
print("Calibrated Parameters:", calibrated_params)

In [ ]:
# 3.Validation
# initial setup

vali_index = []
time_step = 0.1

# Predict for each Vehicle_ID
grouped = test_data.groupby('Vehicle_ID_f')

for vehicle_id, data in grouped:
    # Get the leading vehicle trajectory and the initial state of the following vehicle
    #  YOU CODE HERE
    ####################################################################

    # Initialize the state of the following vehicle
    #  YOU CODE HERE
    ####################################################################

    # Get the actual position trajectory
    #  YOU CODE HERE
    ####################################################################

    # Update the position and velocity of the following vehicle step by step
    for i in range(1, len(data)):
        # Calculate spacing and relative velocity
        #  YOU CODE HERE
        ####################################################################

        # Calculate acceleration using the IDM model
        #  YOU CODE HERE
        ####################################################################

        # Update velocity and position
        #  YOU CODE HERE
        ####################################################################

        # Record the updated velocity and position
        #  YOU CODE HERE
        ####################################################################

    # Calculate the error
    #  YOU CODE HERE
    ####################################################################
    error =
    vali_index.append(error)

    # Plot (to be filled)
    plt.figure(figsize=(10, 6))
    plt.plot(      ,     , label="Leader (to be filled)", color="blue", linestyle='-')
    plt.plot(      ,     , label="Actual (to be filled)", color="blue", linestyle='-')
    plt.plot(      ,     , label="Predicted (to be filled)", color="red", linestyle='--')
    plt.title(f"Vehicle ID: {vehicle_id} - Actual vs Predicted (to be filled)")
    plt.xlabel("Time Step")
    plt.ylabel("         ")
    plt.legend()
    plt.show()

vali_index = np.array(vali_index)
print(np.mean(vali_index))

In [ ]:
# 4. Histogram representing the distribution of the prediction error

#  YOU CODE HERE
####################################################################

# Verify if it belongs to a specific distribution

#  YOU CODE HERE
####################################################################

# It would be great if your report includes:
1. Which GoF is better in IDM model calibration?
2. Which MoP is better in IDM model calibration?
3. Which optimization algorithm is better in IDM model calibration?
4. How to validate the calibrated IDM model?
5. What is prediction error distribution of the estimated model?
6. Plot: Predicted Trajectory vs. True Trajectory, Predicted Speed vs. True Speed.